In [2]:
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [3]:
import tensorflow as tf
x=tf.Variable(3,name="x")
y=tf.Variable(4,name="y")
f=(x*x*y + y + 2)
# Variables dont exist yet until a tensorflow session is initialized

In [4]:
sess = tf.Session()
sess.run(x.initializer) # initialize variable
sess.run(y.initializer)
results = sess.run(f)
print(results)


42


In [5]:
sess.close()

In [6]:
# other option

with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

In [7]:
init = tf.global_variables_initializer() # prepare an unit node

with tf.Session() as sess:
    init.run()
    result= f.eval()
    

In [8]:
# interactive session instead of normal seccion
# doesnt need a with block because once is created is set to the default session

sess = tf.InteractiveSession()
init.run()
result= f.eval()
print(result)
sess.close()

42


In [9]:
x1= tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [10]:
# multiple graphs
graph = tf.Graph()
with graph.as_default():
    x2=tf.Variable(2)
x2.graph is graph

True

In [11]:
# so.. outside the with is not happening anymre
x2.graph is tf.get_default_graph()

False

In [12]:
w=tf.constant(3)
x= w+2
y = x+5
z= x*3

with tf.Session() as sess:
    print(y.eval()) # evaluates x to evaluate y
    print(z.eval()) # evaluate x again to evaluate z

10
15


In [13]:
# to avoid evaluating x and w twince
with tf.Session() as sess:
    y_val, z_val = sess.run([y,z]) # evaluate both just in one graph 
    print(y_val)
    print(z_val)

10
15


# Linear Regression with tf


In [14]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing= fetch_california_housing()
m,n= housing.data.shape # (shape of the dataset)
housing_data_plus_bias = np.c_[np.ones((m,1)),housing.data] # why to add bias??

X=tf.constant(housing_data_plus_bias,dtype=tf.float32,name="X")
y= tf.constant(housing.target.reshape(-1,1),dtype=tf.float32,name="y")# -1 as parameter will insert the automatically detected shape
XT= tf.transpose(X)
theta= tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)),XT),y)

with tf.Session() as sess:
    theta_value = theta.eval()

In [15]:
theta_value

array([[-3.7185181e+01],
       [ 4.3633747e-01],
       [ 9.3952334e-03],
       [-1.0711310e-01],
       [ 6.4479220e-01],
       [-4.0338000e-06],
       [-3.7813708e-03],
       [-4.2348403e-01],
       [-4.3721911e-01]], dtype=float32)

In [16]:
# compare with numpy
X = housing_data_plus_bias
y = housing.target.reshape(-1, 1)
theta_numpy = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(y)

print(theta_numpy)

[[-3.69419202e+01]
 [ 4.36693293e-01]
 [ 9.43577803e-03]
 [-1.07322041e-01]
 [ 6.45065694e-01]
 [-3.97638942e-06]
 [-3.78654265e-03]
 [-4.21314378e-01]
 [-4.34513755e-01]]


# Calculate with gradient descent

first we need to scale the data, because I am already familiarized with standardScaler I will implement that

In [17]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]


In [18]:
reset_graph()
n_epochs=1000
learning_rate=0.01

X= tf.constant(scaled_housing_data_plus_bias,dtype=tf.float32, name="X")
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32, name="y")
theta= tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0),name="theta")
y_pred= tf.matmul(X,theta,name="predicitons") 
error=y_pred - y
mse= tf.reduce_mean(tf.square(error),name="mse")
gradients= tf.gradients(mse,[theta])[0]#gradients = (2/m) *(tf.matmul(tf.transpose(X),error))
training_op = tf.assign(theta,theta - learning_rate*gradients) # this is the operation that will call other variables adn stuff
# because they are nodes, I dont have to evaluate things over and avoer again :D, it kind of looks for the formula and evaluate as needed
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch %100 ==0: #each 100 iterations
            print("Epoch",epoch,"MSE =", mse.eval())
        sess.run(training_op)
    best_theta=theta.eval()

Epoch 0 MSE = 12.408015
Epoch 100 MSE = 0.75519705
Epoch 200 MSE = 0.5420873
Epoch 300 MSE = 0.53317
Epoch 400 MSE = 0.53053814
Epoch 500 MSE = 0.5287964
Epoch 600 MSE = 0.52754897
Epoch 700 MSE = 0.5266498
Epoch 800 MSE = 0.5260011
Epoch 900 MSE = 0.5255332


In [19]:
# a better optimized solution
reset_graph()
n_epochs=1000
learning_rate=0.01

X= tf.constant(scaled_housing_data_plus_bias,dtype=tf.float32, name="X")
y=tf.constant(housing.target.reshape(-1,1),dtype=tf.float32, name="y")
theta= tf.Variable(tf.random_uniform([n+1,1],-1.0,1.0),name="theta")
y_pred= tf.matmul(X,theta,name="predicitons") 
error=y_pred - y
mse= tf.reduce_mean(tf.square(error),name="mse")
##gradients= tf.gradients(mse,[theta])[0]#gradients = (2/m) *(tf.matmul(tf.transpose(X),error))
#training_op = tf.assign(theta,theta - learning_rate*gradients) # this is the operation that will call other variables adn stuff
optimizer= tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)
# because they are nodes, I dont have to evaluate things over and avoer again :D, it kind of looks for the formula and evaluate as needed
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch %100 ==0: #each 100 iterations
            print("Epoch",epoch,"MSE =", mse.eval())
        sess.run(training_op)
    best_theta=theta.eval()

Epoch 0 MSE = 12.408015
Epoch 100 MSE = 0.75519705
Epoch 200 MSE = 0.5420873
Epoch 300 MSE = 0.53317
Epoch 400 MSE = 0.53053814
Epoch 500 MSE = 0.5287964
Epoch 600 MSE = 0.52754897
Epoch 700 MSE = 0.5266498
Epoch 800 MSE = 0.5260011
Epoch 900 MSE = 0.5255332


A different optimizer is the momentum optimizer (that could converge much quicker)
```python
optimizer = tf.train.MomentumOptimizer(learning_rate=learningrate,momentum=0.9)
```

In [20]:
# placeholders to feed data

A= tf.placeholder(tf.float32,shape=(None,3))
B=A+5
with tf.Session() as sess:
    B_eval1=B.eval(feed_dict={A:[[1,2,3]]})
    B_eval2=B.eval(feed_dict={A:[[4,5,6],[7,8,9]]})#Any number of rows
    
print(B_eval1)
print(B_eval2)

[[6. 7. 8.]]
[[ 9. 10. 11.]
 [12. 13. 14.]]


In [21]:
#now make x and y placeholders
X= tf.placeholder(tf.float32,shape=(None,n+1),name="X")
y=tf.placeholder(tf.float32,shape=(None,1),name="y")
# define batch size
batch_size = 100
n_bathches= int(np.ceil(m/batch_size))



In [22]:
# excecution phase
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_bathches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown
    X_batch = scaled_housing_data_plus_bias[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_bathches):
            X_batch, y_batch = fetch_batch(epoch,batch_index,batch_size)
            sess.run(training_op,feed_dict={X: X_batch, y:y_batch})
    best_theta=theta.eval()

# Saving and restoring models

In [23]:
# Saving
reset_graph()

n_epochs = 1000                                                                       # not shown in the book
learning_rate = 0.01                                                                  # not shown

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")            # not shown
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")            # not shown
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")                                      # not shown
error = y_pred - y                                                                    # not shown
mse = tf.reduce_mean(tf.square(error), name="mse")                                    # not shown
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)            # not shown
training_op = optimizer.minimize(mse)                                                 # not shown

init = tf.global_variables_initializer()
# Create server at the end of construction phase
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())        
            # not shown
            # call save whenever
            save_path = saver.save(sess, "/tmp/my_model.ckpt")
        sess.run(training_op)
    
    best_theta = theta.eval()
    # call save whenever
    save_path = saver.save(sess, "/tmp/my_model_final.ckpt")

Epoch 0 MSE = 9.161542
Epoch 100 MSE = 0.7145004
Epoch 200 MSE = 0.56670487
Epoch 300 MSE = 0.5555718
Epoch 400 MSE = 0.54881126
Epoch 500 MSE = 0.5436363
Epoch 600 MSE = 0.53962916
Epoch 700 MSE = 0.5365092
Epoch 800 MSE = 0.53406775
Epoch 900 MSE = 0.5321473


In [24]:
# Restooring
# Create seaver at the end of the construction phase
# Call restores(instead of initializing variables) ath the beggining of the execution phase
with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")
    best_theta_restored = theta.eval() # not shown in the book

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt


In [25]:
best_theta_restored

array([[ 2.0685523 ],
       [ 0.8874027 ],
       [ 0.14401656],
       [-0.3477088 ],
       [ 0.36178365],
       [ 0.00393811],
       [-0.04269556],
       [-0.66145283],
       [-0.6375278 ]], dtype=float32)

Saver saves and reestore all variables, but for more control one can only limit the scope, for example savel that will restore only the theta variable under the name weights:
```python
saver=tf.train.Saver({"weights":theta})
```

# Visualizing the graph
## inside Jupyter

In [26]:
from datetime import datetime
now=datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir="tf_logs" # A different log directory everytime the program is run or else Tensorboard will merge stats from different runs
logdir="{}/run-{}/".format(root_logdir,now)
reset_graph()

n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()
init = tf.global_variables_initializer()
# Create server at the end of construction phase
#saver = tf.train.Saver()
mse_summary=tf.summary.scalar('MSE',mse)
file_writer=tf.summary.FileWriter(logdir,tf.get_default_graph())

n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

with tf.Session() as sess:                                                        # not shown in the book
    sess.run(init)                                                                # not shown

    for epoch in range(n_epochs):                                                 # not shown
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()     

In [27]:
file_writer.close()

# Name scopes

In [28]:
reset_graph()

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")

#########################################################
with tf.name_scope("loss")as scope:
    error = y_pred - y
    mse= tf.reduce_mean(tf.square(error), name="mse")



#################################################
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()

file_writer.flush()
file_writer.close()
print("Best theta:")
print(best_theta)

Best theta:
[[ 2.0703337 ]
 [ 0.8637145 ]
 [ 0.12255152]
 [-0.31211877]
 [ 0.38510376]
 [ 0.00434168]
 [-0.0123295 ]
 [-0.83376896]
 [-0.8030471 ]]


# Modularity

In [29]:
# This is a function that does something, it is not important what right now

def relu(X):
    with tf.name_scope("relu"):
        w_shape=(int(X.get_shape()[1]),1)
        w=tf.Variable(tf.random_normal(w_shape),name="weights")
        b=tf.Variable(0.0,name="bias")
        z= tf.add(tf.matmul(X,w),b,name="z") # I am assuming all of these names are for the tensorgraph
        return tf.maximum(z,0.,name="relu")

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features),name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus,name="output") # add all results from 1 function

file_writer=tf.summary.FileWriter("logs/relu1",tf.get_default_graph())
file_writer.close()

# sharing variables


In [30]:
def relu(X):
    with tf.variable_scope("relu",reuse=True):
        threshold=tf.get_variable("threshold") # reuse existing variable
        w_shape=(int(X.get_shape()[1]),1)
        w=tf.Variable(tf.random_normal(w_shape),name="weights")
        b=tf.Variable(0.0,name="bias")
        z= tf.add(tf.matmul(X,w),b,name="z") # I am assuming all of these names are for the tensorgraph
        return tf.maximum(z,0.,name="relu")

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features),name="X")
with tf.variable_scope("relu"):
    threshold = tf.get_variable("threshold",shape=(),initializer=tf.constant_initializer(0.0)) # creates
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus,name="output") # add all results from 1 function

file_writer=tf.summary.FileWriter("logs/relu1",tf.get_default_graph())
file_writer.close()

In [32]:
# alternative so that the threshold variable doesnt have to be assigned outside of the relu funciton
reset_graph()
def relu(X):
    threshold = tf.get_variable("threshold",shape=(),initializer=tf.constant_initializer(0.0)) # reuse existing variable
    w_shape=(int(X.get_shape()[1]),1)
    w=tf.Variable(tf.random_normal(w_shape),name="weights")
    b=tf.Variable(0.0,name="bias")
    z= tf.add(tf.matmul(X,w),b,name="z") # I am assuming all of these names are for the tensorgraph
    return tf.maximum(z,0.,name="relu")

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features),name="X")
relus = []
for relu_index in range(5):
    with tf.variable_scope("relu",reuse=(relu_index>=1)) as scope:
        relus.append(relu(X))
output = tf.add_n(relus,name="output") # add all results from 1 function

file_writer=tf.summary.FileWriter("logs/relu1",tf.get_default_graph())
file_writer.close()